<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/minimal_circuit_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup
(No need to change anything)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-1dvvssn8
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-1dvvssn8
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit fa287750606075574df2c538058e67d648e2f952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.5 MB/s eta 0:00:00
    

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9100, done.
remote: Counting objects: 100% (1812/1812), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 9100 (delta 1606), reused 1601 (delta 1522), pack-reused 7288
Receiving objects: 100% (9100/9100), 155.60 MiB | 28.21 MiB/s, done.
Resolving deltas: 100% (5501/5501), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [11]:
class Dataset:
    def __init__(self, prompts, pos_dict, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                #     target_token = prompt[targ]
                # else:
                #     target_token = "Ġ" + prompt[targ]
                # target_index = tokens.index(target_token)
                target_index = pos_dict[targ]
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [12]:
pos_dict = {
    'S1': 0,
    'S2': 1,
    'S3': 2,
    'S4': 3,
}

In [13]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'corr': str(i+4),
            'incorr': str(i),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 11)
dataset = Dataset(prompts_list, pos_dict, model.tokenizer, S1_is_first=True)

In [14]:
import random

def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        r = random.randint(50, 100)
        prompt_dict = {
            'S1': str(r),
            'S2': str(r),
            'S3': str(r),
            'S4': str(r),
            'corr': str(r),
            'incorr': str(i+4),
            'text': f"{r} {r} {r} {r}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 11)
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, S1_is_first=True)
prompts_list_2

[{'S1': '61',
  'S2': '61',
  'S3': '61',
  'S4': '61',
  'corr': '61',
  'incorr': '65',
  'text': '61 61 61 61'},
 {'S1': '66',
  'S2': '66',
  'S3': '66',
  'S4': '66',
  'corr': '66',
  'incorr': '70',
  'text': '66 66 66 66'},
 {'S1': '78',
  'S2': '78',
  'S3': '78',
  'S4': '78',
  'corr': '78',
  'incorr': '82',
  'text': '78 78 78 78'},
 {'S1': '75',
  'S2': '75',
  'S3': '75',
  'S4': '75',
  'corr': '75',
  'incorr': '79',
  'text': '75 75 75 75'},
 {'S1': '97',
  'S2': '97',
  'S3': '97',
  'S4': '97',
  'corr': '97',
  'incorr': '101',
  'text': '97 97 97 97'},
 {'S1': '53',
  'S2': '53',
  'S3': '53',
  'S4': '53',
  'corr': '53',
  'incorr': '57',
  'text': '53 53 53 53'},
 {'S1': '58',
  'S2': '58',
  'S3': '58',
  'S4': '58',
  'corr': '58',
  'incorr': '62',
  'text': '58 58 58 58'},
 {'S1': '96',
  'S2': '96',
  'S3': '96',
  'S4': '96',
  'corr': '96',
  'incorr': '100',
  'text': '96 96 96 96'},
 {'S1': '97',
  'S2': '97',
  'S3': '97',
  'S4': '97',
  'corr': '97'

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [15]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [16]:
def mean_ablate_by_lst(lst, model, print_output=True):
    CIRCUIT = {
        "number mover": lst,
        # "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        # "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }

    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

# Ablate the model and compare with original

### try full circuit from repeatLast iter fb

In [17]:
curr_circuit = [(0, 1), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (1, 0), (1, 5), (3, 0), (3, 3), (3, 7), (3, 10), (3, 11), (4, 4), (4, 6), (4, 7), (4, 8), (4, 10), (4, 11), (5, 4), (5, 5), (5, 9), (6, 1), (6, 6), (6, 10), (7, 6), (7, 10), (7, 11), (8, 1), (8, 2), (8, 6), (8, 8), (9, 1), (9, 5), (10, 7), (11, 10)]
mean_ablate_by_lst(curr_circuit, model, print_output=True).item()

Average logit difference (circuit / full) %: 58.1480


58.147972106933594

In [18]:
curr_circuit = [(9, 1)]
mean_ablate_by_lst(curr_circuit, model, print_output=True).item()

Average logit difference (circuit / full) %: -3.3485


-3.3484878540039062

## Prune backwards

In [26]:
# Start with full circuit
curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]
threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

for layer in range(11, -1, -1):  # go thru all heads in a layer first
    for head in range(12):
        # Copying the curr_circuit so we can iterate over one and modify the other
        copy_circuit = curr_circuit.copy()

        # Temporarily removing the current tuple from the copied circuit
        copy_circuit.remove((layer, head))

        new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

        # print((layer,head), new_score)
        # If the result is less than the threshold, remove the tuple from the original list
        if (100 - new_score) < threshold:
            curr_circuit.remove((layer, head))

            print("Removed:", (layer, head))
            print(new_score)
            print("\n")

Removed: (11, 0)
100.0592041015625


Removed: (11, 1)
99.37171936035156


Removed: (11, 2)
99.68282318115234


Removed: (11, 3)
99.3035659790039


Removed: (11, 4)
101.24650573730469


Removed: (11, 5)
101.45158386230469


Removed: (11, 6)
101.5175552368164


Removed: (11, 7)
101.33053588867188


Removed: (11, 8)
100.53523254394531


Removed: (11, 9)
99.48712921142578


Removed: (11, 10)
97.55806732177734


Removed: (11, 11)
98.188720703125


Removed: (10, 0)
97.4887466430664


Removed: (10, 2)
101.01201629638672


Removed: (10, 3)
101.50897216796875


Removed: (10, 4)
102.61394500732422


Removed: (10, 5)
103.2564468383789


Removed: (10, 6)
103.1583023071289


Removed: (10, 7)
102.306884765625


Removed: (10, 8)
102.56729888916016


Removed: (10, 9)
102.76387023925781


Removed: (10, 10)
103.2691879272461


Removed: (10, 11)
102.70503997802734


Removed: (9, 0)
103.00797271728516


Removed: (9, 2)
103.20918273925781


Removed: (9, 3)
106.37710571289062


Removed: (9, 4)
104.711685180

In [27]:
mean_ablate_by_lst(curr_circuit, model, print_output=True)

Average logit difference (circuit / full) %: 105.1309


tensor(105.1309, device='cuda:0')

In [28]:
backw_3 = curr_circuit.copy()
backw_3

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 8),
 (0, 9),
 (0, 10),
 (1, 0),
 (1, 5),
 (1, 7),
 (2, 0),
 (2, 2),
 (2, 9),
 (3, 0),
 (4, 2),
 (4, 4),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (5, 0),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (6, 1),
 (6, 3),
 (6, 4),
 (6, 6),
 (6, 8),
 (6, 10),
 (7, 10),
 (7, 11),
 (8, 11),
 (9, 1),
 (10, 1)]

In [29]:
len(backw_3)

36

Now try 10% threshold:

In [30]:
def find_circuit_backw(threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    # Start with full circuit
    curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(11, -1, -1):  # go thru all heads in a layer first
        for head in range(12):
            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("Removed:", (layer, head))
                print(new_score)
                print("\n")

    return curr_circuit

In [31]:
curr_circuit = find_circuit_backw(10)

Removed: (11, 0)
100.0592041015625


Removed: (11, 1)
99.37171936035156


Removed: (11, 2)
99.68282318115234


Removed: (11, 3)
99.3035659790039


Removed: (11, 4)
101.24650573730469


Removed: (11, 5)
101.45158386230469


Removed: (11, 6)
101.5175552368164


Removed: (11, 7)
101.33053588867188


Removed: (11, 8)
100.53523254394531


Removed: (11, 9)
99.48712921142578


Removed: (11, 10)
97.55806732177734


Removed: (11, 11)
98.188720703125


Removed: (10, 0)
97.4887466430664


Removed: (10, 1)
95.24443817138672


Removed: (10, 2)
98.80593872070312


Removed: (10, 3)
99.32583618164062


Removed: (10, 4)
100.39134979248047


Removed: (10, 5)
100.99724578857422


Removed: (10, 6)
100.89531707763672


Removed: (10, 7)
100.18833923339844


Removed: (10, 8)
100.45863342285156


Removed: (10, 9)
100.64905548095703


Removed: (10, 10)
101.12189483642578


Removed: (10, 11)
100.5857162475586


Removed: (9, 0)
100.8254165649414


Removed: (9, 2)
100.99942779541016


Removed: (9, 3)
104.07704925

In [32]:
backw_10 = curr_circuit.copy()
backw_10

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 8),
 (0, 9),
 (0, 10),
 (1, 0),
 (1, 5),
 (1, 8),
 (2, 0),
 (2, 2),
 (2, 9),
 (3, 0),
 (3, 3),
 (3, 7),
 (3, 8),
 (4, 2),
 (4, 4),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (5, 0),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 8),
 (6, 1),
 (6, 4),
 (6, 6),
 (6, 8),
 (6, 9),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 6),
 (7, 8),
 (7, 10),
 (7, 11),
 (9, 1)]

In [33]:
mean_ablate_by_lst(curr_circuit, model, print_output=True)

Average logit difference (circuit / full) %: 94.6093


tensor(94.6093, device='cuda:0')

In [34]:
len(backw_10)

43

20%:

In [35]:
%%capture
curr_circuit = find_circuit_backw(20)

In [36]:
backw_20 = curr_circuit.copy()
backw_20

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 9),
 (0, 10),
 (0, 11),
 (1, 0),
 (1, 5),
 (1, 7),
 (1, 8),
 (2, 0),
 (2, 2),
 (2, 7),
 (2, 9),
 (3, 0),
 (3, 3),
 (4, 4),
 (4, 6),
 (4, 7),
 (4, 9),
 (4, 10),
 (4, 11),
 (5, 0),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 8),
 (6, 1),
 (6, 6),
 (6, 9),
 (6, 10),
 (7, 10),
 (7, 11),
 (9, 1)]

In [37]:
mean_ablate_by_lst(curr_circuit, model, print_output=True)

Average logit difference (circuit / full) %: 80.5078


tensor(80.5078, device='cuda:0')

In [38]:
len(backw_20)

35

### set diffs of the three perf lvls

In [39]:
set(backw_3) - set(backw_10)

{(0, 0), (1, 7), (6, 3), (6, 10), (8, 11), (10, 1)}

In [40]:
set(backw_10) - set(backw_3)

{(1, 8),
 (3, 3),
 (3, 7),
 (3, 8),
 (4, 6),
 (5, 2),
 (5, 8),
 (6, 9),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 6),
 (7, 8)}

In [41]:
set(backw_3) - set(backw_20)

{(0, 0), (0, 8), (4, 2), (4, 8), (6, 3), (6, 4), (6, 8), (8, 11), (10, 1)}

In [42]:
set(backw_10) - set(backw_20)

{(0, 8),
 (3, 7),
 (3, 8),
 (4, 2),
 (4, 8),
 (5, 2),
 (6, 4),
 (6, 8),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 6),
 (7, 8)}

In [43]:
mean_ablate_by_lst(backw_20, model, print_output=True)

Average logit difference (circuit / full) %: 80.5078


tensor(80.5078, device='cuda:0')

In [44]:
mean_ablate_by_lst(backw_20 + [(10, 2)], model, print_output=True)

Average logit difference (circuit / full) %: 79.1801


tensor(79.1801, device='cuda:0')

In [45]:
mean_ablate_by_lst([x for x in backw_20 if x != (9, 1)], model, print_output=True)

Average logit difference (circuit / full) %: 33.4869


tensor(33.4869, device='cuda:0')

In [46]:
mean_ablate_by_lst([x for x in backw_20 if x != (9, 1)] + [(10, 2)], model, print_output=True)

Average logit difference (circuit / full) %: 39.1113


tensor(39.1113, device='cuda:0')

### set diff w repeatLast and repeatFirstAll circs

In [52]:
repeatFirstAll_backw_3 = [(0, 1), (0, 9), (1, 0), (1, 5), (2, 2), (2, 9), (2, 10), (3, 0), (3, 3), (3, 6), (3, 7), (4, 4), (4, 8), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 8), (6, 1), (6, 3), (6, 4), (6, 6), (6, 9), (6, 10), (7, 1), (7, 2), (7, 6), (7, 7), (7, 10), (7, 11), (8, 0), (8, 1), (8, 6), (8, 8), (8, 11), (9, 1), (9, 9), (10, 1), (10, 2), (11, 8), (11, 9), (11, 10)]
repeatFirstAll_backw_10 = [(0, 1), (0, 9), (1, 0), (1, 5), (1, 6), (2, 2), (2, 8), (2, 9), (3, 0), (3, 2), (3, 3), (3, 7), (3, 8), (3, 10), (4, 4), (5, 1), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 10), (6, 0), (6, 1), (6, 3), (6, 4), (6, 6), (6, 9), (6, 10), (6, 11), (7, 0), (7, 6), (7, 8), (7, 10), (7, 11), (8, 0), (8, 1), (8, 6), (8, 8), (8, 11), (9, 1), (10, 2)]
repeatFirstAll_backw_20 = [(0, 1), (0, 9), (1, 0), (1, 5), (1, 6), (2, 2), (2, 8), (2, 9), (2, 10), (3, 0), (3, 2), (3, 3), (3, 7), (3, 10), (4, 4), (4, 10), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 8), (5, 10), (6, 1), (6, 3), (6, 4), (6, 6), (6, 9), (6, 10), (7, 2), (7, 6), (7, 7), (7, 10), (7, 11), (8, 0), (8, 6), (8, 8), (8, 11), (9, 1)]

In [53]:
mean_ablate_by_lst(repeatFirstAll_backw_3, model, print_output=True)

Average logit difference (circuit / full) %: 9.5653


tensor(9.5653, device='cuda:0')

In [55]:
mean_ablate_by_lst(repeatFirstAll_backw_10, model, print_output=True)

Average logit difference (circuit / full) %: 4.4688


tensor(4.4688, device='cuda:0')

In [54]:
mean_ablate_by_lst(repeatFirstAll_backw_20, model, print_output=True)

Average logit difference (circuit / full) %: 27.7859


tensor(27.7859, device='cuda:0')

In [48]:
repLast_backw_3 = [(0, 1), (0, 3), (0, 5), (0, 7), (0, 9), (0, 10), (1, 0), (1, 4), (1, 5), (2, 2), (2, 8), (2, 9), (3, 0), (3, 2), (3, 3), (3, 7), (4, 4), (4, 7), (4, 10), (5, 1), (5, 3), (5, 4), (5, 5), (5, 6), (5, 8), (5, 9), (6, 1), (6, 4), (6, 6), (6, 10), (6, 11), (7, 6), (7, 10), (7, 11), (8, 0), (8, 5), (8, 6), (8, 8), (9, 1), (10, 7)]
set(backw_3) - set(repLast_backw_3)

{(0, 0),
 (0, 2),
 (0, 8),
 (1, 7),
 (2, 0),
 (4, 2),
 (4, 8),
 (4, 9),
 (5, 0),
 (6, 3),
 (6, 8),
 (8, 11),
 (10, 1)}

In [58]:
mean_ablate_by_lst(repLast_backw_3, model, print_output=True)

Average logit difference (circuit / full) %: 29.2602


tensor(29.2602, device='cuda:0')

In [49]:
set(repLast_backw_3) - set(backw_3)

{(0, 5),
 (0, 7),
 (1, 4),
 (2, 8),
 (3, 2),
 (3, 3),
 (3, 7),
 (5, 1),
 (5, 8),
 (5, 9),
 (6, 11),
 (7, 6),
 (8, 0),
 (8, 5),
 (8, 6),
 (8, 8),
 (10, 7)}

In [50]:
set(backw_3) - set(repeatFirstAll_backw_3)

{(0, 0),
 (0, 2),
 (0, 3),
 (0, 8),
 (0, 10),
 (1, 7),
 (2, 0),
 (4, 2),
 (4, 7),
 (4, 9),
 (4, 10),
 (5, 0),
 (6, 8)}

In [51]:
set(repeatFirstAll_backw_3) - set(backw_3)

{(2, 10),
 (3, 3),
 (3, 6),
 (3, 7),
 (5, 1),
 (5, 2),
 (5, 8),
 (6, 9),
 (7, 1),
 (7, 2),
 (7, 6),
 (7, 7),
 (8, 0),
 (8, 1),
 (8, 6),
 (8, 8),
 (9, 9),
 (10, 2),
 (11, 8),
 (11, 9),
 (11, 10)}

## Prune forwards

In [ ]:
# # Start with full circuit
# curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]
# threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

# for layer in range(0, 12):
#     for head in range(12):
#         # Copying the curr_circuit so we can iterate over one and modify the other
#         copy_circuit = curr_circuit.copy()

#         # Temporarily removing the current tuple from the copied circuit
#         copy_circuit.remove((layer, head))

#         new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

#         # print((layer,head), new_score)
#         # If the result is less than the threshold, remove the tuple from the original list
#         if (100 - new_score) < threshold:
#             curr_circuit.remove((layer, head))

#             print("Removed:", (layer, head))
#             print(new_score)
#             print("\n")

## Prune fwds-backwds iteratively

In [19]:
def find_circuit_forw(curr_circuit=None, threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    if curr_circuit == []:
        # Start with full circuit
        curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(0, 12):
        for head in range(12):
            if (layer, head) not in curr_circuit:
                continue

            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

            # print((layer,head), new_score)
            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("\nRemoved:", (layer, head))
                print(new_score)

    return curr_circuit, new_score

In [20]:
def find_circuit_backw(curr_circuit=None, threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    if curr_circuit == []:
        # Start with full circuit
        curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(11, -1, -1):  # go thru all heads in a layer first
        for head in range(12):
            if (layer, head) not in curr_circuit:
                continue

            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("\nRemoved:", (layer, head))
                print(new_score)

    return curr_circuit, new_score

### iter fwd backw, threshold 3

In [21]:
threshold = 3
curr_circuit = []
prev_score = 100
new_score = 0
iter = 1
while prev_score != new_score:
    print('\nfwd prune, iter ', str(iter))
    # track changes in circuit as for some reason it doesn't work with scores
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_forw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    print('\nbackw prune, iter ', str(iter))
    # prev_score = new_score # save old score before finding new one
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_backw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    iter += 1


fwd prune, iter  1

Removed: (0, 3)
98.36941528320312

Removed: (0, 5)
101.85221862792969

Removed: (0, 6)
108.80953216552734

Removed: (0, 7)
98.24569702148438

Removed: (0, 11)
104.70988464355469

Removed: (1, 1)
106.21051025390625

Removed: (1, 2)
105.51274108886719

Removed: (1, 3)
105.42396545410156

Removed: (1, 4)
106.2643814086914

Removed: (1, 6)
104.33666229248047

Removed: (1, 7)
99.28598022460938

Removed: (1, 9)
104.02457427978516

Removed: (1, 10)
105.2890625

Removed: (1, 11)
102.27257537841797

Removed: (2, 1)
101.98612976074219

Removed: (2, 3)
98.50780487060547

Removed: (2, 4)
99.79397583007812

Removed: (2, 5)
99.86790466308594

Removed: (2, 8)
99.03162384033203

Removed: (2, 9)
99.62406921386719

Removed: (2, 10)
98.79183197021484

Removed: (2, 11)
100.31877136230469

Removed: (3, 1)
101.15858459472656

Removed: (3, 2)
103.99312591552734

Removed: (3, 3)
97.90157318115234

Removed: (3, 4)
97.06743621826172

Removed: (3, 5)
97.63529968261719

Removed: (3, 9)
97.391

In [22]:
fb_3 = curr_circuit.copy()
fb_3

[(0, 1),
 (0, 2),
 (0, 8),
 (0, 9),
 (0, 10),
 (1, 0),
 (1, 5),
 (2, 0),
 (2, 2),
 (2, 6),
 (2, 7),
 (3, 0),
 (3, 11),
 (4, 4),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (5, 0),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (6, 1),
 (6, 6),
 (6, 10),
 (7, 10),
 (7, 11),
 (8, 8),
 (8, 9),
 (8, 11),
 (9, 1),
 (11, 10)]

In [24]:
mean_ablate_by_lst(fb_3, model, print_output=True)

Average logit difference (circuit / full) %: 97.2114


tensor(97.2114, device='cuda:0')

In [25]:
mean_ablate_by_lst(fb_3 + [(6, 9)], model, print_output=True)

Average logit difference (circuit / full) %: 94.0516


tensor(94.0516, device='cuda:0')

#### compare

In [56]:
set(backw_3) - set(fb_3)

{(0, 0), (0, 3), (1, 7), (2, 9), (4, 2), (6, 3), (6, 4), (6, 8), (10, 1)}

In [57]:
set(fb_3) - set(backw_3)

{(2, 6), (2, 7), (3, 11), (4, 6), (8, 8), (8, 9), (11, 10)}

### iter fwd backw, threshold 20

In [ ]:
# threshold = 20
# curr_circuit = []
# prev_score = 100
# new_score = 0
# iter = 1
# while prev_score != new_score:
#     print('\nfwd prune, iter ', str(iter))
#     # track changes in circuit as for some reason it doesn't work with scores
#     old_circuit = curr_circuit.copy() # save old before finding new one
#     curr_circuit, new_score = find_circuit_forw(curr_circuit=curr_circuit, threshold=threshold)
#     if curr_circuit == old_circuit:
#         break
#     print('\nbackw prune, iter ', str(iter))
#     # prev_score = new_score # save old score before finding new one
#     old_circuit = curr_circuit.copy() # save old before finding new one
#     curr_circuit, new_score = find_circuit_backw(curr_circuit=curr_circuit, threshold=threshold)
#     if curr_circuit == old_circuit:
#         break
#     iter += 1

In [ ]:
# curr_circuit

## Prune backwds-fwds iteratively

### iter fwd backw, threshold 3

In [ ]:
threshold = 3
curr_circuit = []
prev_score = 100
new_score = 0
iter = 1
while prev_score != new_score:
    print('\nbackw prune, iter ', str(iter))
    # prev_score = new_score # save old score before finding new one
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_backw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    print('\nfwd prune, iter ', str(iter))
    # track changes in circuit as for some reason it doesn't work with scores
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_forw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    iter += 1

In [ ]:
bf_3 = curr_circuit.copy()
bf_3

#### compare

In [ ]:
len(bf_3)

In [ ]:
len(fb_3)

In [ ]:
set(backw_3) - set(bf_3)

In [ ]:
set(bf_3) - set(backw_3)

In [ ]:
set(fb_3) - set(bf_3)

In [ ]:
set(bf_3) - set(fb_3)